In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
# 读取TSV文件
tsv_file = "filereport_read_run_PRJEB1787.tsv"  # 替换为你的TSV文件名
df = pd.read_csv(tsv_file, sep='\t')
df1=pd.read_csv(tsv_file, sep='\t')

In [3]:

sample_accessions = df['sample_accession']

# 定义一个函数，用于下载Sample Accession的信息
def download_sample_info(sample_accession):
    url = f'https://www.ebi.ac.uk/ena/browser/api/xml/{sample_accession}'
    response = requests.get(url)
    return response.text


In [4]:

# 下载所有Sample Accession的信息，并将其存储在一个字典中
sample_info_dict = {}
for sample_accession in sample_accessions:
    sample_info = download_sample_info(sample_accession)
    sample_info_dict[sample_accession] = sample_info

# 如果需要，可以将其保存为CSV文件
df = pd.DataFrame(sample_info_dict.items(), columns=['Sample Accession', 'Sample Information'])
df.to_csv('tara_sample_information.csv', index=False)

ConnectionError: ('Connection aborted.', TimeoutError(10060, '由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。', None, 10060, None))

整理结果

In [5]:
# 读取CSV文件
data = pd.read_csv("sample_information.csv")

# 初始化一个空的DataFrame，用于存放提取的数据
columns = ["Sample Accession", "Sample Title", "Scientific Name", "Description", "Collection Date", "Longitude", "Latitude", "Depth (m)"]
extracted_data = pd.DataFrame(columns=columns)

# 遍历CSV中的每一行，解析XML内容并提取所需信息
for index, row in data.iterrows():
    sample_accession = row["Sample Accession"]
    xml_string = row["Sample Information"]

    # 解析XML内容
    root = ET.fromstring(xml_string)

    sample = root.find("SAMPLE")
    title = sample.find("TITLE").text
    scientific_name = sample.find("SAMPLE_NAME").find("SCIENTIFIC_NAME").text
    description = sample.find("DESCRIPTION").text

    attributes = sample.find("SAMPLE_ATTRIBUTES")
    
    collection_date = None
    longitude = None
    latitude = None
    depth = None

    for attribute in attributes.findall("SAMPLE_ATTRIBUTE"):
        tag = attribute.find("TAG").text
        value = attribute.find("VALUE").text

        if tag == "collection date":
            collection_date = value
        elif tag == "geographic location (longitude)":
            longitude = value
        elif tag == "geographic location (latitude)":
            latitude = value
        elif tag == "geographic location (depth)":
            depth = value

    # 将提取的信息添加到新的DataFrame中
    extracted_data = extracted_data.append({
        "Sample Accession": sample_accession,
        "Sample Title": title,
        "Scientific Name": scientific_name,
        "Description": description,
        "Collection Date": collection_date,
        "Longitude": longitude,
        "Latitude": latitude,
        "Depth (m)": depth
    }, ignore_index=True)

# 将新的DataFrame保存为CSV文件


C:\Users\DuYih\AppData\Local\Temp\ipykernel_1412\2441961242.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append({
C:\Users\DuYih\AppData\Local\Temp\ipykernel_1412\2441961242.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append({
C:\Users\DuYih\AppData\Local\Temp\ipykernel_1412\2441961242.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append({
C:\Users\DuYih\AppData\Local\Temp\ipykernel_1412\2441961242.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append({
C:\Users\DuYih\AppData\Local

In [9]:

extracted_data.insert(loc=0, column='new_column', value=df1['run_accession'])

In [10]:

extracted_data.to_csv("tara_meta.csv", index=False)
